<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=08ac6292c97d25462d4a1940d3e97c782f2b03de2d9db559c73ce42dee009d0d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 13:46:24
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 1.22 L (0.86%)
Current PnL: -25.98 L (-16.57%)
CY Booked + Current PnL: -11.69 L (-7.45%)
-------------------
Total profit:  1.52 L
Total loss:  -27.50 L
-------------------
Total Booked + Current PnL: 15.29 L (11.78%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.99%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.36 L (63.85%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,0.88,-19.69,24.63,0.09,19935.0,-19839.0,80938.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.34,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,1.13,13.59,6.87,21.40,20714.0,36083.0,301509.0,5.44,67.0,X-LC,5.20,12.0,1.74,2.11,20.78,XY25,NTT,FMCG
51,MASFIN,398.61,2.29,-1.52,23.93,22.05,23091.0,-1485.0,96495.0,-15.09,55.0,H-SC,6.58,164.0,-0.06,0.67,40.26,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.89,6.60,12.28,19.69,25101.0,12663.0,204405.0,-3.84,47.0,X-MC,2.13,78.0,0.50,1.43,13.85,XY25,NTT,AC
44,ITC,452.00,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.00,0.88,57.06,MH,ATH,POWER
10,BAJAJHLDNG,14451.52,3.92,2.69,26.62,30.03,36458.0,3586.0,136956.0,-0.67,52.0,X-LC,12.13,35.0,0.10,0.96,11.42,X40,ATH,FINANCE
23,DIXON,18931.72,3.74,-18.71,55.09,26.07,87423.0,-36530.0,158691.0,-60.51,33.0,X-MC,16.05,56.0,-0.42,1.11,3.74,X40N,ATH,IT
66,SHALBY,327.00,3.48,-22.20,60.79,25.10,85674.0,-40208.0,140935.0,936.68,50.0,M-SC,9.13,250.0,-0.47,0.99,12.95,XY24,NTT,HEALTHCARE
41,INDUSINDBK,1800.00,3.15,-35.92,107.37,32.89,53122.0,-27730.0,49476.0,-710.84,62.0,L-MC,6.71,259.0,-0.52,0.35,36.25,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.00,-4.19,-64.45,181.52,0.07,88509.0,-88417.0,48760.0,1686.62,42.0,L-SC,4.35,268.0,-1.00,0.34,21.27,OX40N,NTT,JEWELLERY
85,WHIRLPOOL,2270.00,-2.41,-28.78,155.43,81.92,127080.0,-33038.0,81760.0,-58.28,26.0,M-SC,23.79,236.0,-0.26,0.57,0.00,XR,NTT,DURABLES
56,QUESS,424.00,-2.10,-30.64,105.96,42.84,47773.0,-19920.0,45086.0,-54.82,40.0,X-SC,31.63,83.0,-0.42,0.32,2.00,XY24,NTT,MISC
39,INDIAMART,4810.62,-1.46,-4.85,117.26,106.72,137608.0,-5983.0,117353.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.39,AR,ATH,MISC
19,COFFEEDAY,80.00,-1.41,-37.54,116.57,35.27,82677.0,-42624.0,70925.0,-51.71,55.0,L-SC,7.00,270.0,-0.52,0.50,73.59,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.0,0.88,57.06,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,1.41,-6.00,121.19,107.91,168190.0,-8862.0,138782.0,-23.39,42.0,M-SC,11.63,216.0,-0.05,0.97,3.48,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.78,-18.84,23.24,0.02,32924.0,-32889.0,141670.0,115.96,39.0,M-SC,8.36,234.0,-1.00,0.99,23.23,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.88,-19.69,24.63,0.09,19935.0,-19839.0,80938.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.34,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.87,-21.27,44.13,13.47,93692.0,-57357.0,212310.0,-67.73,55.0,H-SC,6.49,158.0,-0.61,1.48,7.95,XY24,NTT,PAINTS
68,SIS,528.00,-0.26,-23.58,58.68,21.26,49791.0,-26181.0,84851.0,2002.59,51.0,H-SC,4.50,166.0,-0.53,0.59,14.64,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.00,0.88,57.06,MH,ATH,POWER
86,WIPRO,420.00,-0.82,7.26,60.61,72.27,110154.0,12294.0,181742.0,-7.89,53.0,M-LC,5.19,99.0,0.11,1.27,13.88,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.82,7.26,60.61,72.27,110154.0,12294.0,181742.0,-7.89,53.0,M-LC,5.19,99.0,0.11,1.27,13.88,XR,NTT,IT
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.00,0.88,57.06,MH,ATH,POWER
6,ASIANTILES,137.00,-0.13,-1.21,83.89,81.67,77748.0,-1132.0,92678.0,8344.44,65.0,L-SC,15.99,271.0,-0.01,0.65,79.47,XR,NTT,CERAMICS
39,INDIAMART,4810.62,-1.46,-4.85,117.26,106.72,137608.0,-5983.0,117353.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.39,AR,ATH,MISC
26,FINCABLES,1641.55,1.41,-6.00,121.19,107.91,168190.0,-8862.0,138782.0,-23.39,42.0,M-SC,11.63,216.0,-0.05,0.97,3.48,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.12,-19.43,92.96,55.47,182221.0,-47265.0,196021.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.44,X40N,ATH,FINANCE
35,HONAUT,58357.33,-0.05,-19.00,76.95,43.32,101512.0,-30952.0,131920.0,-30.40,29.0,X-SC,7.22,90.0,-0.30,0.92,1.29,X40N,ATH,ELECTRICAL
23,DIXON,18931.72,3.74,-18.71,55.09,26.07,87423.0,-36530.0,158691.0,-60.51,33.0,X-MC,16.05,56.0,-0.42,1.11,3.74,X40N,ATH,IT
8,AWL,485.00,-0.83,-24.68,103.78,53.50,235882.0,-74461.0,227290.0,-63.50,35.0,X-MC,6.96,58.0,-0.32,1.59,0.95,XY24,NTT,FMCG
28,GLAXO,3466.20,1.43,-2.92,43.33,39.15,83828.0,-5820.0,193464.0,-22.01,35.0,X-MC,10.83,60.0,-0.07,1.35,25.23,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.47,-17.41,51.97,25.51,79875.0,-32400.0,153695.0,-5.62,42.0,H-LC,0.99,49.0,-0.41,1.07,13.26,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,0.97,5.60,14.36,20.77,35508.0,13123.0,247267.0,-3.53,53.0,X-MC,1.57,62.0,0.37,1.73,12.91,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,1.68,-4.80,27.25,21.14,58978.0,-10907.0,216432.0,-11.49,43.0,X-MC,1.63,74.0,-0.18,1.51,20.19,XY24,NTT,PAINTS
21,DABUR,735.0,2.07,-1.09,45.29,43.70,111124.0,-2706.0,245362.0,-9.24,57.0,X-MC,1.99,72.0,-0.02,1.72,13.80,XY24,BTT,FMCG
44,ITC,452.0,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.12,-19.43,92.96,55.47,182221.0,-47265.0,196021.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.44,X40N,ATH,FINANCE
12,BATAINDIA,2096.00,0.59,-39.13,122.13,35.20,95651.0,-50351.0,78319.0,2.67,36.0,X-SC,17.36,92.0,-0.53,0.55,0.59,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.21,-27.06,125.04,64.15,217032.0,-64381.0,173570.0,-57.19,35.0,X-SC,5.65,82.0,-0.30,1.21,0.61,XY24,BTT,CEMENT
20,COLPAL,3726.84,0.74,-21.46,80.16,41.51,165819.0,-56505.0,206860.0,-6.53,36.0,X-MC,8.39,61.0,-0.34,1.45,0.74,XY25,ATH,FMCG
54,PAGEIND,51605.08,0.90,-10.92,43.81,28.11,78606.0,-21985.0,179425.0,-35.49,35.0,X-MC,15.20,55.0,-0.28,1.25,0.90,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.10,-30.64,105.96,42.84,47773.0,-19920.0,45086.0,-54.82,40.0,X-SC,31.63,83.0,-0.42,0.32,2.00,XY24,NTT,MISC
60,RELAXO,1176.00,1.11,-47.57,190.48,52.31,144978.0,-69048.0,76112.0,-44.15,46.0,X-SC,7.38,91.0,-0.48,0.53,1.88,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.59,-39.13,122.13,35.20,95651.0,-50351.0,78319.0,2.67,36.0,X-SC,17.36,92.0,-0.53,0.55,0.59,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.19,-2.73,27.55,24.08,34021.0,-3460.0,123490.0,-10.62,44.0,X-SC,7.68,89.0,-0.10,0.86,16.36,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.05,-19.00,76.95,43.32,101512.0,-30952.0,131920.0,-30.40,29.0,X-SC,7.22,90.0,-0.30,0.92,1.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-0.85,-11.49,36.37,20.71,121762.0,-43451.0,334786.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.86,X40,ATH,IT
42,INFY,1972.00,-0.07,10.32,21.69,34.24,74867.0,32278.0,345166.0,-14.21,53.0,X-LC,7.33,2.0,0.43,2.41,19.45,X40,NTT,IT
77,TMPV,600.00,1.17,-24.98,63.93,22.98,108334.0,-56440.0,169458.0,-21.86,57.0,X-LC,5.85,3.0,-0.52,1.18,6.58,XY24,NTT,AUTO
83,VBL,671.64,1.06,-1.10,37.18,35.67,116137.0,-3477.0,312365.0,-12.80,62.0,X-LC,2.91,4.0,-0.03,2.18,12.60,X40N,ATH,FMCG
44,ITC,452.00,0.70,-1.13,12.05,10.78,28631.0,-2715.0,237603.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.05,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,3.15,-35.92,107.37,32.89,53122.0,-27730.0,49476.0,-710.84,62.0,L-MC,6.71,259.0,-0.52,0.35,36.25,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.13,-1.21,83.89,81.67,77748.0,-1132.0,92678.0,8344.44,65.0,L-SC,15.99,271.0,-0.01,0.65,79.47,XR,NTT,CERAMICS
51,MASFIN,398.61,2.29,-1.52,23.93,22.05,23091.0,-1485.0,96495.0,-15.09,55.0,H-SC,6.58,164.0,-0.06,0.67,40.26,XR,ATH,FINANCE
14,BSOFT,831.70,2.31,-21.43,91.90,50.79,101565.0,-30136.0,110517.0,0.89,55.0,H-SC,11.15,151.0,-0.30,0.77,28.68,XR,ATH,IT
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.00,0.88,57.06,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,4.91,0.06,46.87,46.95,58761.0,73.0,125370.0,-7.63,68.0,H-SC,9.41,154.0,0.00,0.88,57.06,MH,ATH,POWER
6,ASIANTILES,137.00,-0.13,-1.21,83.89,81.67,77748.0,-1132.0,92678.0,8344.44,65.0,L-SC,15.99,271.0,-0.01,0.65,79.47,XR,NTT,CERAMICS
14,BSOFT,831.70,2.31,-21.43,91.90,50.79,101565.0,-30136.0,110517.0,0.89,55.0,H-SC,11.15,151.0,-0.30,0.77,28.68,XR,ATH,IT
76,TITAGARH,1548.00,2.53,-15.20,71.86,45.74,147580.0,-36806.0,205371.0,13.00,66.0,H-SC,4.81,174.0,-0.25,1.44,33.57,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,1.53,10.02,18.82,30.73,40791.0,19736.0,216741.0,-15.76,68.0,X-MC,4.28,75.0,0.48,1.52,24.06,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.13
2,50,75.06


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.23
MC    30.13
LC    25.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.48
X40      23.47
X40N     12.64
XY25      9.93
XR        9.62
AR        8.63
OX40N     7.36
SR        0.98
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.72
H-SC    23.61
X-LC    22.26
M-SC    11.28
X-SC     7.85
H-MC     4.71
L-SC     1.49
M-MC     1.35
M-LC     1.27
H-LC     1.07
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.12,-6.54,40.80
IT,12.53,-17.33,76.33
FINANCE,10.44,-15.55,64.61
MISC,6.99,-29.78,82.87
ELECTRICAL,5.82,-12.70,53.97
PAINTS,5.69,-13.32,30.15
INSURANCE,4.70,-1.04,35.61
PHARMA,3.96,-3.57,36.18
AUTO,2.82,-30.11,75.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3132262.0,21
AR,1337391.0,10
XR,1284440.0,13
X40,1037759.0,15
X40N,876791.0,9
OX40N,739676.0,10
XY25,387804.0,7
SR,280826.0,2
MH,58761.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3600666.0,25
M-SC,1444602.0,15
X-MC,1433453.0,16
X-LC,880894.0,13
X-SC,795556.0,8
H-MC,404645.0,3
L-SC,248934.0,3
M-LC,110154.0,1
H-LC,79875.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1236587.0      6
           AR         915854.0      5
M-SC       XY24       814512.0      6
H-SC       XR         781298.0      7
X-MC       X40        487683.0      7
           XY24       405984.0      3
X-LC       X40        368818.0      6
X-MC       X40N       348866.0      4
H-SC       OX40N      327340.0      4
M-SC       OX40N      323827.0      5
X-SC       X40N       315472.0      3
           XY24       298826.0      3
H-SC       SR         280826.0      2
H-MC       AR         214840.0      2
X-LC       X40N       212453.0      2
X-MC       XY25       190920.0      2
H-MC       XY24       189805.0      1
X-LC       XY24       186548.0      2
X-SC       X40        181258.0      2
M-SC       XR         179441.0      2
L-SC       XR         160425.0      2
M-SC       AR         126822.0      2
X-LC       XY25       113075.0      3
M-LC       XR         110154.0      1
L-SC       OX40N       88509.0      1
H-LC       AR          79875.0      1
H-SC       MH          58761.0      1
L-MC       XR          53122.0      1
M-MC       XY25        52028.0      1
L-LC       XY25        31781.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
